In [1]:
DL = '/Users/damiansp/Downloads'

In [2]:
!rm $DL/*

In [3]:
from datetime import datetime
import os

import numpy as np
import pandas as pd

In [4]:
SYMBOLS = [
    'AAPL', 'ABMD', 'ABNB', 'ADYEY', 'AMD', 'AMZN', 'ANET', 'APPH',
    'AXON', 'BROS', 'BYND', 'CHWY', 'COIN', 'COUR', 'CRWD', 'DDOG',
    'DKNG', 'DOCS', 'DOCU', 'DUOL', 'EDIT', 'ETSY', 'FLGT',
    'FUV', 'FVRR', 'GH', 'GMED', 'GOOG', 'INTG', 'ISRG', 'JD', 'LSPD',
    'LULU', 'MDB', 'MELI', 'MNST', 'MSFT', 'MTCH', 'NVCR', 'NVDA',
    'NYT', 'OKTA', 'OPEN', 'PANW', 'PINS', 'PTON', 'QDEL', 'RBLX',
    'RDFN', 'RGEN', 'RIVN', 'ROKU', 'RVLV', 'SE', 'SHOP', 'SNOW',
    'SPLK', 'SQ', 'TASK', 'TDOC', 'TEAM', 'TREX', 'TTD', 'TTWO', 'TWLO',
    'TWTR', 'UPST', 'VEEV', 'WK', 'XPEV', 'ZEN', 'ZNGA']

In [5]:
!ls $DL

Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips(2).csv
Holdings - Damian Satterthwaite-Phillips.csv


In [6]:
def check_symbols(df):
    all_present = True
    for symbol in df.index.unique():
        if symbol not in SYMBOLS:
            print(f'{symbol} not in SYMBOLS')
            all_present = False
    if all_present:
        print('All clear')

In [7]:
def convert_value(s):
    return round(float(s.replace('$', '').replace(',', '')))

In [8]:
binder = pd.DataFrame(index=SYMBOLS)
binder.head()

""
AAPL
ABMD
ABNB
ADYEY
AMD


### E*Trade

In [15]:
DONGMEI = False

In [16]:
if DONGMEI:
    et_df = (
        pd.read_csv(
            f'{DL}/2022-04-29.csv', index_col=0, skiprows=1
        )[['Market Value']]
        .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['Portfolio', 'Cash'], inplace=True)
else:
    et_df = (
        pd.read_csv(
            f'{DL}/Positions.csv', index_col=0, skiprows=1
        )[['Market Value']]
        .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['FNRG', 'Portfolio', 'Cash'], inplace=True)
    #et_df.ET = et_df.ET.apply(lambda x: round(float(x)))
et_df.head()

,ET
AAPL,526.24
ABMD,459.58
ABNB,497.45
ADYEY,2650.24
AMD,1386.6899999999998


In [17]:
#et_df.drop(index=['98986CONT'], inplace=True)
#t_df.head()

In [18]:
et_df.loc[et_df.ET == '--']
#et_df.loc['LSPD', 'ET'] = 20.31

,ET


In [19]:
check_symbols(et_df)

All clear


In [20]:
out = pd.concat([binder, et_df], axis=1).fillna(0)
out.ET = out.ET.astype(float).apply(lambda x: int(round(x)))
out.head()

,ET
AAPL,526
ABMD,460
ABNB,497
ADYEY,2650
AMD,1387


### Fidelity

In [21]:
TODAY = datetime.now().date()
TODAY
today = datetime.strftime(TODAY, '%b-%d-%Y')
filename = f'Portfolio_Positions_{today}.csv'
filename

'Portfolio_Positions_Jun-17-2022.csv'

In [22]:
!ls $DL

Holdings - Damian Satterthwaite-Phillips(0).csv
Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips(2).csv
Portfolio_Positions_Jun-17-2022.csv
Positions.csv
mw.csv


In [23]:
#df_raw = pd.read_csv(f'{DL}/{filename.replace("-02", "-03")}')
df_raw = pd.read_csv(f'{DL}/{filename}')
df_raw.head()

,Account Number,Account Name,Symbol,Description,Quantity,Last Price,Last Price Change,Current Value,Today's Gain/Loss Dollar,Today's Gain/Loss Percent,Total Gain/Loss Dollar,Total Gain/Loss Percent,Percent Of Account,Cost Basis,Cost Basis Per Share,Type
0,218520300,ROLLOVER IRA,SPAXX**,FIDELITY GOVERNMENT MONEY MARKET,151.51,$1.00,$0.00,$151.51,NaN,NaN,NaN,NaN,0.45%,NaN,NaN,Cash
1,218520300,ROLLOVER IRA,AAPL,APPLE INC,16.00,$131.56,+$1.50,$2104.96,+$24.00,+1.15%,-$319.20,-13.17%,6.28%,$2424.16,$151.51,Cash
2,218520300,ROLLOVER IRA,ABMD,ABIOMED INC COM USD0.01,3.00,$229.79,+$5.06,$689.37,+$15.18,+2.25%,-$78.23,-10.20%,2.06%,$767.60,$255.87,Cash
3,218520300,ROLLOVER IRA,ABNB,AIRBNB INC COM CL A,3.00,$99.49,+$6.23,$298.47,+$18.69,+6.68%,-$129.78,-30.31%,0.89%,$428.25,$142.75,Cash
4,218520300,ROLLOVER IRA,ADYEY,ADYEN NV UNSPON ADS EACH REP 0.01 ORD SHS,1.00,$13.12,+$0.604,$13.12,+$0.60,+4.82%,-$1.97,-13.06%,0.04%,$15.09,$15.09,Cash


In [24]:
df = df_raw[['Account Name', 'Symbol', 'Current Value']].dropna()
df['Current Value'] = df['Current Value'].apply(convert_value)
df['Current Value'] = df['Current Value'].fillna(0).astype(int)
df = df[((df.Symbol != 'SPAXX**') & (df.Symbol != 'Pending Activity'))]
df.head()

,Account Name,Symbol,Current Value
1,ROLLOVER IRA,AAPL,2105
2,ROLLOVER IRA,ABMD,689
3,ROLLOVER IRA,ABNB,298
4,ROLLOVER IRA,ADYEY,13
5,ROLLOVER IRA,AMD,1387


In [25]:
accounts = {account_name: df[df['Account Name'] == account_name]
            for account_name in df['Account Name'].unique()}
for acct in accounts.keys():
    accounts[acct].index = accounts[acct].Symbol
    accounts[acct] = (
        accounts[acct]
        .rename(columns={'Current Value': acct})
        .drop(columns=['Account Name', 'Symbol']))
    
accounts['ROLLOVER IRA'].head()

,ROLLOVER IRA
Symbol,
AAPL,2105
ABMD,689
ABNB,298
ADYEY,13
AMD,1387


In [26]:
for k, v in accounts.items():
    print(k)
    check_symbols(v)
    print()

ROLLOVER IRA
All clear

ROTH IRA
All clear

SIMPLE IRA
All clear



In [27]:
for v in accounts.values():
    out = pd.concat([out, v], axis=1)
out.fillna(0, inplace=True)
out = out.astype(int)
out.head()

,ET,ROLLOVER IRA,ROTH IRA,SIMPLE IRA
AAPL,526,2105,132,0
ABMD,460,689,460,0
ABNB,497,298,0,0
ADYEY,2650,13,1627,0
AMD,1387,1387,1387,245


In [28]:
out.to_csv(f'{DL}/clean.csv')

In [29]:
!open $DL/clean.csv

In [ ]:
!rm $DL/$filename $DL/Positions.csv $DL/clean.csv
!ls $DL

### MarketWatch

In [9]:
!mv "$DL/Holdings - Damian Satterthwaite-Phillips.csv" "$DL/Holdings - Damian Satterthwaite-Phillips(0).csv"
!ls $DL

Holdings - Damian Satterthwaite-Phillips(0).csv
Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips(2).csv


In [10]:
dfs = [binder]
for i, f in enumerate(sorted(os.listdir(DL))):
    print(f)
    if f.startswith('Holdings'):
        mw = pd.read_csv(f'{DL}/{f}')[['Symbol', 'Value']]
        mw.rename(columns={'Value': f'Value_{i}'}, inplace=True)
        mw.index = mw.Symbol
        mw.drop('Symbol', axis=1, inplace=True)
        mw[f'Value_{i}'] = mw[f'Value_{i}'].apply(convert_value)
        dfs.append(mw)
mw = pd.concat(dfs, axis=1).fillna(0).astype(int)
mw.head()

.DS_Store
Holdings - Damian Satterthwaite-Phillips(0).csv
Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips(2).csv


,Value_1,Value_2,Value_3
AAPL,658,2105,1447
ABMD,919,689,0
ABNB,597,497,995
ADYEY,945,1535,1168
AMD,734,1795,1224


In [11]:
check_symbols(mw)

All clear


In [12]:
#out = pd.concat([binder, mw], axis=1).fillna(0).astype(int)
mw.head()

,Value_1,Value_2,Value_3
AAPL,658,2105,1447
ABMD,919,689,0
ABNB,597,497,995
ADYEY,945,1535,1168
AMD,734,1795,1224


In [13]:
mw.to_csv(f'{DL}/mw.csv')

In [14]:
!open $DL/mw.csv

In [ ]:
!rm $DL/Holdings*.csv $DL/mw.csv
!ls $DL

In [ ]:
!rm $DL/*